## Webtretho(웹트레토) 커뮤니티 데이터 수집 ver.3

In [ ]:
# ================================================================
# 🧭 Webtretho 로그인 & 다중 키워드 크롤러 (tqdm + 로그 + 키워드별 저장 + 합본)
# ================================================================
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time, os, re
from tqdm import tqdm   # ✅ 진행률 표시

# ================================================================
# 1) 기본 설정
# ================================================================

scroll_num = 50

USERNAME = ""
PASSWORD = ""

MAIN_URL = "https://www.webtretho.vn/"
LOGIN_URL = "https://www.webtretho.vn/dang-nhap"

# housing_keywords_vi = ["ẩm mốc nhà ở", "máy lọc không khí", "mùi hôi nhà bếp"]
KEYWORDS = [
    "Nhà thông minh",                 # 스마트홈
    "IoT",                            # IoT
    "Thiết bị gia dụng",              # 가전제품
    "Căn hộ", "Chung cư",             # 아파트
    "Nhà bếp",                        # 주방
    "Thông gió",                      # 환기
    "Nấm mốc",                        # 곰팡이
    "Mùi hôi",                        # 냄새
    "Vệ sinh",                        # 위생
    "Vi khuẩn",                       # 세균
    "Côn trùng", "Sâu bọ", "Muỗi",    # 벌레 / 해충 / 모기
    "Mất điện",                       # 정전
    "Rò rỉ nước",                     # 누수
    "Nước máy", "Hệ thống nước",      # 수도
    "Bảo quản",                       # 보관
    "Khử mùi",                        # 탈취
    "Đá", "Làm đá",                   # 얼음
    "Tiết kiệm điện",                 # 절전
    "Tiết kiệm năng lượng",           # 절약
    "Hóa đơn tiền điện",              # 전기요금
    "Máy lọc không khí",              # 공기청정기
    "Máy điều hòa",                   # 에어컨
    "Tủ lạnh",                        # 냉장고
    "Máy lọc nước",                   # 정수기
    "Máy giặt",                       # 세탁기
    "Máy sấy",                        # 건조기
    "Máy hút ẩm",                     # 제습기
    "Trợ lý", "Người giúp việc",      # 도우미
    "LG",                             # LG
    "Chuyên gia",                     # 전문가
    "Công ty dịch vụ"                 # 서비스업체
]

SAVE_DIR = "webtretho_results"
os.makedirs(SAVE_DIR, exist_ok=True)
CSV_PATH_ALL = os.path.join(SAVE_DIR, "webtretho_articles_all.csv")
LOG_PATH = os.path.join(SAVE_DIR, "error_log.txt")

# 키워드별 파일명 안전 슬러그
def slugify(text):
    text = text.strip().lower()
    text = re.sub(r"\s+", "_", text)
    text = re.sub(r"[^a-z0-9_\-가-힣ぁ-んァ-ン一-龥ấầẩẫậắằẳẵặếềểễệơờởỡợôốồổỗộưứừửữựđ]", "", text)
    return text[:60] if len(text) > 60 else text

# ================================================================
# 2) 에러 로그 저장
# ================================================================
def log_error(keyword, url, err_msg):
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {keyword} | {url} | {err_msg}\n")

# ================================================================
# 3) 로그인
# ================================================================
def webtretho_login(driver):
    try:
        driver.get(LOGIN_URL)
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, "input[type='text']").send_keys(USERNAME)
        driver.find_element(By.CSS_SELECTOR, "input[type='password']").send_keys(PASSWORD)
        driver.find_element(
            By.CSS_SELECTOR,
            "#__next > div > div.jsx-1971079523 > div > div > div > div > div > div:nth-child(4) > div > button"
        ).click()
        time.sleep(3)
        if "dang-nhap" not in driver.current_url:
            print("✅ 로그인 성공!")
            driver.get(MAIN_URL); time.sleep(2)
        else:
            print("⚠️ 로그인 실패 — 아이디/비번 확인 필요")
    except Exception as e:
        print("🚨 로그인 중 에러:", e)
        log_error("LOGIN", LOGIN_URL, str(e))

# ================================================================
# 4) 실행
# ================================================================
driver = wb.Chrome()
driver.maximize_window()

print("🌐 메인 페이지 접속 중...")
driver.get(MAIN_URL); time.sleep(2)

print("🔐 로그인 시도 중...")
webtretho_login(driver)
print("📄 키워드 기반 데이터 수집 시작...")

all_data = []  # 합본 용

# ================================================================
# 5) 키워드별 수집 루프 (+키워드별 저장)
# ================================================================
for keyword in KEYWORDS:
    print(f"\n🔎 현재 키워드: {keyword}")
    driver.get(MAIN_URL); time.sleep(1)

    # (1) 검색
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="desktop-layout"]/div[1]/div/div/div[3]/div/div/div[2]/input'))
        )
        search_box.click(); time.sleep(0.2)
        search_box.clear()
        search_box.send_keys(keyword + Keys.ENTER)
        time.sleep(2)
    except Exception as e:
        print(f"❌ [{keyword}] 검색 실패:", e)
        log_error(keyword, "SEARCH", str(e))
        continue

    # (2) ‘Xem thêm’ 클릭 반복 (필요시 range 숫자 조절)
    for i in tqdm(range(scroll_num), desc=f"🔽 [{keyword}] 스크롤 진행중", ncols=80):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            xem_them_btn = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.ga-seemore-searched-posts"))
            )
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", xem_them_btn)
            time.sleep(0.3)
            driver.execute_script("arguments[0].click();", xem_them_btn)
            time.sleep(1.6)
        except Exception:
            break


    # (3) 목록 URL/날짜
    try:
        aTags = driver.find_elements(
            By.CSS_SELECTOR,
            "div.tw-flex.tw-text-left.tw-flex-row.tw-items-start a.tw-block"
        )
        url_list = [a.get_attribute("href") for a in aTags]

        date_elems = driver.find_elements(
            By.CSS_SELECTOR,
            "div.tw-text-xs.text-gray-blue.tw-mt-1\\.5"
        )
        date_list = [d.text.strip() for d in date_elems]
        url_date_list = list(zip(url_list, date_list))
        print(f"🔗 [{keyword}] 목록 {len(url_date_list)}건")
    except Exception as e:
        print(f"❌ [{keyword}] URL 수집 실패:", e)
        log_error(keyword, "URL_COLLECT", str(e))
        continue

    # (4) 상세 수집 + 진행률
    data_this_kw = []
    for href, date in tqdm(url_date_list, desc=f"📘 [{keyword}] 본문 수집", ncols=80):
        try:
            driver.get(href); time.sleep(1)
            try:
                title = driver.find_element(By.CSS_SELECTOR, 'h1.text-wrap.tw-font-bold.text-primary').text
            except:
                title = None
            try:
                content = driver.find_element(By.ID, 'post-content').text
            except:
                content = None
            data_this_kw.append([keyword, title, content, date, href])
        except Exception as e:
            log_error(keyword, href, str(e))
            continue

    # (5) 키워드별 CSV 저장
    try:
        df_kw = pd.DataFrame(data_this_kw, columns=['keyword', 'title', 'content', 'date', 'url'])
        # 중복 제거(같은 키워드 내에서)
        if not df_kw.empty:
            # 결측/공백 정리
            df_kw = df_kw.fillna("")
            # 제목/본문 모두 비면 제거
            df_kw = df_kw[~((df_kw['title'].str.strip() == "") & (df_kw['content'].str.strip() == ""))].copy()
            # URL 중복 제거
            df_kw.drop_duplicates(subset=["url"], inplace=True)


            fname = f"webtretho_{slugify(keyword)}.csv"
            path_kw = os.path.join(SAVE_DIR, fname)
            df_kw.to_csv(path_kw, index=False, encoding="utf-8-sig")
            print(f"💾 [{keyword}] {len(df_kw)}건 저장 → {path_kw}")
            # 합본에도 누적
            all_data.extend(df_kw.values.tolist())
        else:
            print(f"ℹ️ [{keyword}] 수집된 데이터 없음")
    except Exception as e:
        print(f"❌ [{keyword}] 키워드별 저장 실패:", e)
        log_error(keyword, "SAVE_PER_KEYWORD", str(e))
        # 그래도 합본 누적은 진행(비어있으면 영향 없음)

# ================================================================
# 6) 합본 저장
# ================================================================
driver.close()

try:
    df_final = pd.DataFrame(all_data, columns=['keyword', 'title', 'content', 'date', 'url'])
    if not df_final.empty:
        df_final.drop_duplicates(subset=["url"], inplace=True)
        df_final.to_csv(CSV_PATH_ALL, index=False, encoding='utf-8-sig')
        print(f"\n🎯 전체 합본 저장 완료! 총 {len(df_final)}건 → {CSV_PATH_ALL}")
    else:
        print("\n⚠️ 합칠 데이터가 없습니다.")
except Exception as e:
    print("❌ 합본 저장 실패:", e)
    log_error("ALL", "SAVE_ALL", str(e))

print(f"🪵 에러 로그: {LOG_PATH}")
